# Using KGTK query to do interesting queries in Wikidata
This notebook shows use cases of interesting queries on Wikidata that can be done using the KGTK query command (aka Kypher), and that cannot be done using the public Wikidata SPARQL endpoint

The notebook has a preamble to set up environment variables to access the relevant files

In [1]:
# Parameters

# Folder on local machine where to create the output and temporary folders
output_path = "/Users/pedroszekely/Downloads/kypher"

# The names of the output and temporary folders
output_folder = "wd-workshop"
temp_folder = "temp.wd-workshop"

# The location of input Wikidata files
wikidata_folder = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/"
# wikidata_folder = "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/"
# The wikidata_os files can be downloaded from https://drive.google.com/drive/folders/1V6oAQKmwQ4LJnrBai-uv5gHWphFSCt50?usp=sharing

wikidata_dbpedia_folder = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-dbpedia"

# Location of the cache database for kypher
cache_path = "/Users/pedroszekely/Downloads/kypher/temp.novartis"
# cache_path = "/Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v4/wikidata.sqlite3.db"
# Whether to delete the cache database
delete_database = False

# shortcuts to commands
kgtk = "time kgtk --debug"
# kgtk = "kgtk --debug"
# kgtk = "kgtk"

In [2]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import altair as alt

import papermill as pm

In [3]:
# The names of files in the KGTK Wikidata distirbution that we will use in this notebook.
file_names = {
    "claims": "claims.tsv.gz",
    "quantity": "claims.quantity.tsv.gz",
    "time": "claims.time.tsv.gz",
    "label": "labels.en.tsv.gz",
    "alias": "aliases.en.tsv.gz",
    "description": "descriptions.en.tsv.gz",
    "item": "claims.wikibase-item.tsv.gz",
    "external_id": "claims.external-id.tsv.gz",
    "qualifiers": "qualifiers.tsv.gz",
    "sitelinks": "sitelinks.tsv.gz",
    "qualifiers_time": "qualifiers.time.tsv.gz",
    "property_datatypes": "metadata.property.datatypes.tsv.gz",
    "isa": "derived.isa.tsv.gz",
    "p279star": "derived.P279star.tsv.gz",
    "p279": "derived.P279.tsv.gz",
    "p31": "derived.P31.tsv.gz",
    "dwd_isa": "derived.dwd_isa.tsv.gz"
}

# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

os.environ['WIKIDATA'] = wikidata_folder
kgtk_environment_variables.append('WIKIDATA')

for key, value in file_names.items():
    variable = key.upper()
    os.environ[variable] = wikidata_folder + value
    kgtk_environment_variables.append(variable)

os.environ["WD2DB"] = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-dbpedia/wikidata_to_dbpedia_edge_file.tsv.gz"
kgtk_environment_variables.append("WD2DB")


# KGTK creates a SQLite database to index the knowledge graph.
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
kgtk_environment_variables.append('STORE')

# We will create many temporary files, so set up a folder for outputs and one for the temporary files.
os.environ['TEMP'] = "{}/{}".format(output_path, temp_folder) 
os.environ['OUT'] = "{}/{}".format(output_path, output_folder) 
kgtk_environment_variables.append('TEMP')
kgtk_environment_variables.append('OUT')

# Envronment variables with shortcuts to the commands we use often
os.environ['kgtk'] = kgtk
# Use for debugging, but careful as it causes import to dataframes to break
os.environ['kypher'] = "time kgtk --debug query --graph-cache " + os.environ['STORE']
# os.environ['kypher'] = "kgtk query --graph-cache " + os.environ['STORE']
kgtk_environment_variables.append('kgtk')
kgtk_environment_variables.append('kypher')

# We'll save the current working directory so we can call into other example notebooks later
os.environ["EXAMPLES_DIR"] = os.getcwd()
kgtk_environment_variables.append('EXAMPLES_DIR')

kgtk_environment_variables.sort()
for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

ALIAS: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/aliases.en.tsv.gz"
CLAIMS: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/claims.tsv.gz"
DESCRIPTION: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/descriptions.en.tsv.gz"
DWD_ISA: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/derived.dwd_isa.tsv.gz"
EXAMPLES_DIR: "/Users/pedroszekely/Documents/GitHub/kgtk-at-2021-wikidata-workshop"
EXTERNAL_ID: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/claims.external-id.tsv.gz"
ISA: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/derived.isa.tsv.gz"
ITEM: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/claims.wikibase-item.tsv.gz"
LABEL: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/labels.en.tsv.gz"
OUT: "/Users/pedroszekely/Downloads/kypher/wd-workshop"
P279: "/Volumes/GoogleDrive/Shar

In [4]:
%cd {output_path}

/Users/pedroszekely/Downloads/kypher


Define the shortcuts for Kypher so that import the relevant files into the Kypher index and define shortcuts to make the queries nicer to write

In [5]:
!$kypher \
-i "$ITEM" --as items \
-i "$TIME" --as time \
-i "$P31" --as p31 \
-i "$P279" --as p279 \
-i "$LABEL" --as labels \
-i "$ALIAS" --as aliases \
-i "$P279STAR" --as p279star \
-i "$QUALIFIERS" --as qualifiers \
-i "$DESCRIPTION" --as descriptions \
-i "$EXTERNAL_ID" --as external_ids \
-i "$WD2DB" --as wd2db \
--limit 10

[2021-07-18 11:03:51 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
id	node1	label	node2	rank	node2;wikidatatype
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508	normal	wikibase-item
P10-P1855-Q15075950-7eff6d65-0	P10	P1855	Q15075950	normal	wikibase-item
P10-P1855-Q4504-a69d2c73-0	P10	P1855	Q4504	normal	wikibase-item
P10-P1855-Q69063653-c8cdb04c-0	P10	P1855	Q69063653	normal	wikibase-item
P10-P1855-Q7378-555592a4-0	P10	P1855	Q7378	normal	wikibase-item
P10-P2302-Q21502404-d012aef4-0	P10	P2302	Q21502404	normal	wikibase-item
P10-P2302-Q21510851-5224fe0b-0	P10	P2302	Q21510851	normal	wikibase-item
P10-P2302-Q21510852-dde2f0ce-0	P10	P2302	Q21510852	normal	wikibase-item
P10-P2302-Q52004125-d0288d06-0	P10	P2302	Q52004125	normal	wikibase-item
P10-P2302-Q53869507-974ce3b1-0	P10	P2302	Q53869507	normal	wikibase-item
        1.13 real         0.80 user         0.20 

## Retrieve large amounts of data from Wikidata

John is doing research on the popularity of first names to improve his entity resolution algorithm for people. He sees that Wikidata contains about 9 million people, so he wants to get the distribution of counts of first names from Wikidata. He writes a SPARQL query, but it times out, so he downloads the Wikidata KGTK files on his laptop and writes a kypher query. The query retrieves all instances of human (Q5), gets their frst names using the P735 property and return the counts.

John thinks he will want to do additional analysis on the data, so chooses standard KGTK names for the headers to generate the data as a KGTK graph that then he can use as input to other KGTK commands.

In [45]:
!$kypher -i items -i p31 -i labels \
--match '\
    p31: (person)-[]->(:Q5), \
    items: (person)-[:P735]->(given_name), \
    labels: (given_name)-[]->(given_name_label)' \
--return 'distinct given_name as node1, count(given_name) as node2, given_name_label as `node1;label`, "count_names" as label' \
--order-by 'node2 desc' \
-o "$OUT"/given-names.tsv

[2021-07-18 16:33:35 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c2."node2" "_aLias.node1", count(graph_8_c2."node2") "_aLias.node2", graph_4_c3."node2" "_aLias.node1;label", ? "_aLias.label"
     FROM graph_2 AS graph_2_c1
     INNER JOIN graph_4 AS graph_4_c3, graph_8 AS graph_8_c2
     ON graph_2_c1."node1" = graph_8_c2."node1"
        AND graph_8_c2."node2" = graph_4_c3."node1"
        AND graph_2_c1."node2" = ?
        AND graph_8_c2."label" = ?
     GROUP BY "_aLias.node1"
     ORDER BY "_aLias.node2" DESC
  PARAS: ['count_names', 'Q5', 'P735']
---------------------------------------------
      381.73 real        52.79 user        92.72 sys


In [47]:
!wc "$OUT"/given-names.tsv 

   53253  216696 1988754 /Users/pedroszekely/Downloads/kypher/wd-workshop/given-names.tsv


John takes a peek at the file to make sure he got the headers correcly: an edge from the q-node to the count, using `count_names` as the property, and including the `label` of `node1` so he can read the data. John sees that his name is by far the most popular name in Wikidata, and gets the information he needs to fine tune his entity resolution algorithms. 

In [48]:
!head "$OUT"/given-names.tsv | column -ts $'\t'

node1      node2   node1;label   label
Q4925477   120416  'John'@en     count_names
Q12344159  74235   'William'@en  count_names
Q4927937   59298   'Robert'@en   count_names
Q16428906  57107   'Thomas'@en   count_names
Q677191    52568   'James'@en    count_names
Q18057751  49005   'David'@en    count_names
Q2958359   44735   'Charles'@en  count_names
Q2793400   40987   'Peter'@en    count_names
Q1249148   40149   'Richard'@en  count_names


John gets curious and wants to know whether the popularity of names depends of time, so modifies his query to partition the data by people's year of birth.

In [49]:
!$kypher -i items -i time -i p31 -i labels \
--match '\
    p31: (person)-[]->(:Q5), \
    items: (person)-[:P735]->(given_name), \
    time: (person)-[:P569]->(date_of_birth), \
    labels: (given_name)-[]->(given_name_label)' \
--return 'distinct given_name as node1, kgtk_date_year(date_of_birth) as year, count(given_name) as node2, given_name_label as `node1;label`, "count_names_yearly" as label' \
--order-by 'given_name, cast(year, integer), node2 desc' \
-o "$OUT"/given-names.year.tsv

[2021-07-18 16:41:09 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c2."node2" "_aLias.node1", kgtk_date_year(graph_14_c3."node2") "_aLias.year", count(graph_8_c2."node2") "_aLias.node2", graph_4_c4."node2" "_aLias.node1;label", ? "_aLias.label"
     FROM graph_14 AS graph_14_c3
     INNER JOIN graph_2 AS graph_2_c1, graph_4 AS graph_4_c4, graph_8 AS graph_8_c2
     ON graph_2_c1."node1" = graph_14_c3."node1"
        AND graph_2_c1."node1" = graph_8_c2."node1"
        AND graph_8_c2."node2" = graph_4_c4."node1"
        AND graph_14_c3."label" = ?
        AND graph_2_c1."node2" = ?
        AND graph_8_c2."label" = ?
     GROUP BY "_aLias.node1", "_aLias.year"
     ORDER BY graph_8_c2."node2" ASC, CAST("_aLias.year" AS integer) ASC, "_aLias.node2" DESC
  PARAS: ['count_names_yearly', 'P569', 'Q5', 'P735']
---------------------------------------------
      451.92 real        82.87 user        91.36 sys


In [10]:
!wc "$OUT"/given-names.year.tsv 

  882179 3549576 25649279 /Users/pedroszekely/Downloads/kypher/wd-workshop/given-names.year.tsv


John takea a quick peek at the file to verify that the headers are correct.

In [54]:
!head -5 "$OUT"/given-names.year.tsv | column -ts $'\t'

node1     year  node2  node1;label      label
Q1000387  1798  1      'Ferdinanda'@en  count_names_yearly
Q1000387  1849  1      'Ferdinanda'@en  count_names_yearly
Q1000387  1868  1      'Ferdinanda'@en  count_names_yearly
Q1000387  1870  1      'Ferdinanda'@en  count_names_yearly


John heard anecdotaly that Jessica had become a popular name in the late 90s and greps for Jessica in the file. 

In [12]:
!grep "'Jessica'" "$OUT"/given-names.year.tsv | tail -15

Q630846	1995	58	'Jessica'@en
Q630846	1996	41	'Jessica'@en
Q630846	1997	27	'Jessica'@en
Q630846	1998	23	'Jessica'@en
Q630846	1999	23	'Jessica'@en
Q630846	2000	51	'Jessica'@en
Q630846	2001	18	'Jessica'@en
Q630846	2002	18	'Jessica'@en
Q630846	2003	11	'Jessica'@en
Q630846	2004	6	'Jessica'@en
Q630846	2005	2	'Jessica'@en
Q630846	2009	1	'Jessica'@en
Q630846	2011	1	'Jessica'@en
Q630846	2014	1	'Jessica'@en
Q630846	2016	2	'Jessica'@en


John realizes that he needs to normalize the counts of names by the number of people born in each year. He wonders whether he can do it in one kypher query, but takes the easy way out and writes a simple query to get the counts of people born each year. He can do this faster than he can think of a complex query to get the final result in one go.

In [50]:
!$kypher -i time -i p31 \
--match ' \
    p31: (person)-[]->(:Q5), \
    time: (person)-[:P569]->(date_of_birth)' \
--return 'kgtk_date_year(date_of_birth) as node1, count(person) as node2, "count_people_born" as label' \
-o "$TEMP"/human.count.year.tsv

[2021-07-18 16:48:41 query]: SQL Translation:
---------------------------------------------
  SELECT kgtk_date_year(graph_14_c2."node2") "_aLias.node1", count(graph_2_c1."node1") "_aLias.node2", ? "_aLias.label"
     FROM graph_14 AS graph_14_c2
     INNER JOIN graph_2 AS graph_2_c1
     ON graph_2_c1."node1" = graph_14_c2."node1"
        AND graph_14_c2."label" = ?
        AND graph_2_c1."node2" = ?
     GROUP BY "_aLias.node1"
  PARAS: ['count_people_born', 'P569', 'Q5']
---------------------------------------------
      130.18 real        37.81 user        34.96 sys


John is happy that KGTK accepts literals as subjects of triples because here the subjects (`node1`) are years.

In [51]:
!head "$TEMP"/human.count.year.tsv

node1	node2	label
1	105	count_people_born
2	5	count_people_born
3	9	count_people_born
4	8	count_people_born
5	11	count_people_born
6	10	count_people_born
7	7	count_people_born
8	5	count_people_born
9	9	count_people_born


John knows he is almost there. He needs to get the names from the `given-names.year.tsv` file, and needs to pick out the year from the qualifier he put on the edge using the syntax to get the attributes of edges `[r {year: the_year}]`. He computes the fraction of people with each name and multiplies by 100,000 so that the numbers are not so tiny and easier to read. John also gets the labels of the q-nodes from the attribute he put on `node1` so tha the doesn't have to join with the `labels.tsv` file again.

In [71]:
!$kypher -i "$OUT"/given-names.year.tsv -i "$TEMP"/human.count.year.tsv \
--match ' \
    names: (given_name {label: given_name_label})-[r {year: the_year}]->(count_names), \
    year: (the_year)-[]->(count_people)' \
--return 'given_name as node1, "normalized_count_names_yearly" as label, cast(count_names, float) * 10000 / cast(count_people, float) as node2, the_year as year, given_name_label as `node1;label`' \
--order-by 'given_name, cast(the_year, integer), node2 desc' \
-o "$OUT"/given-names.year.normalized.tsv

[2021-07-18 16:59:11 query]: SQL Translation:
---------------------------------------------
  SELECT graph_34_c1."node1" "_aLias.node1", ? "_aLias.label", (CAST(graph_34_c1."node2" AS float) * (? / CAST(graph_35_c2."node2" AS float))) "_aLias.node2", graph_35_c2."node1" "_aLias.year", graph_34_c1."node1;label" "_aLias.node1;label"
     FROM graph_34 AS graph_34_c1
     INNER JOIN graph_35 AS graph_35_c2
     ON graph_35_c2."node1" = graph_34_c1."year"
        AND graph_34_c1."node1;label" = graph_34_c1."node1;label"
        AND graph_34_c1."year" = graph_35_c2."node1"
     ORDER BY graph_34_c1."node1" ASC, CAST(graph_35_c2."node1" AS integer) ASC, "_aLias.node2" DESC
  PARAS: ['normalized_count_names_yearly', 10000]
---------------------------------------------
        4.63 real         4.30 user         0.30 sys


In [77]:
!head "$OUT"/given-names.year.normalized.tsv | column -ts $'\t'

node1     label                          node2               year  node1;label
Q1000387  normalized_count_names_yearly  2.207505518763797   1798  'Ferdinanda'@en
Q1000387  normalized_count_names_yearly  1.1767474699929394  1849  'Ferdinanda'@en
Q1000387  normalized_count_names_yearly  0.6827336655970506  1868  'Ferdinanda'@en
Q1000387  normalized_count_names_yearly  0.6334726973267453  1870  'Ferdinanda'@en
Q1000387  normalized_count_names_yearly  0.536711034778875   1888  'Ferdinanda'@en
Q1000433  normalized_count_names_yearly  1.0892059688487092  1852  'Bud'@en
Q1000433  normalized_count_names_yearly  0.9004141905276427  1858  'Bud'@en
Q1000433  normalized_count_names_yearly  0.6082355087890031  1881  'Bud'@en
Q1000433  normalized_count_names_yearly  0.5845218611176058  1882  'Bud'@en


John greps the normalized file again. Jessica was not a popular name in the 60s and began to get popular in the late 70s. John satisfied his curiosity. The popularity of names is time dependent, but for now, John will work to integrate the aggregate data into his entity resolution algorithm.

In [76]:
!grep "'Jessica'" "$OUT"/given-names.year.normalized.tsv | tail -50

Q630846	normalized_count_names_yearly	1.5782828282828283	1960	'Jessica'@en
Q630846	normalized_count_names_yearly	1.6427682992654478	1961	'Jessica'@en
Q630846	normalized_count_names_yearly	0.6936416184971098	1962	'Jessica'@en
Q630846	normalized_count_names_yearly	2.2975301550832854	1963	'Jessica'@en
Q630846	normalized_count_names_yearly	2.7612232218872963	1964	'Jessica'@en
Q630846	normalized_count_names_yearly	3.534651365553644	1965	'Jessica'@en
Q630846	normalized_count_names_yearly	3.7685601587820012	1966	'Jessica'@en
Q630846	normalized_count_names_yearly	2.2576760987357014	1967	'Jessica'@en
Q630846	normalized_count_names_yearly	3.7844383893430216	1968	'Jessica'@en
Q630846	normalized_count_names_yearly	4.271034846619601	1969	'Jessica'@en
Q630846	normalized_count_names_yearly	5.083022704168078	1970	'Jessica'@en
Q630846	normalized_count_names_yearly	8.008208413623965	1971	'Jessica'@en
Q630846	normalized_count_names_yearly	5.099569086412198	1972	'Jessica'@en
Q630846	normalized_count_names

## Analytics on full Wikidata

Jessica is working with John on the entity resolution algorithm and her job is to use the number of instances of each class in Wikidata as a feature. The query that Jessica needs to write is simple as she just needs to count the number of instances of each class, summing up over the instances of all subclasses. She knows that there are over 1 million classes in Wikidata (entities with a P279 property), so she knows it will not run on the public SPARQL endpint. Jessica gets the SQLite database from John so that she does not have to wait the 2 or so hours to load it on her laptop, writes the query and goes for lunch as she knows it will take a while for it to run.

In [22]:
!$kypher -i p31 -i p279star \
--match '\
    p31: (entity)-[]->(class), \
    p279star: (class)-[]->(super_class)' \
--return 'distinct super_class as node1, count(distinct entity) as node2, "entity_count" as label' \
--order-by 'node2 desc, node1' \
-o "$OUT"/class.count.tsv.gz

[2021-07-18 13:17:26 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_5_c2."node2" "_aLias.node1", count(DISTINCT graph_2_c1."node1") "_aLias.node2", ? "_aLias.label"
     FROM graph_2 AS graph_2_c1
     INNER JOIN graph_5 AS graph_5_c2
     ON graph_2_c1."node2" = graph_5_c2."node1"
     GROUP BY "_aLias.node1"
     ORDER BY "_aLias.node2" DESC, "_aLias.node1" ASC
  PARAS: ['entity_count']
---------------------------------------------
     4743.27 real      4354.24 user       299.54 sys


After coming back from lunch, the file is ready, it contains data for 75K classes, she figures that the other classes don't have instances.

In [32]:
!zcat < "$OUT"/class.count.tsv.gz | wc

   75195  225585 1863081


In [80]:
!zcat < "$OUT"/class.count.tsv.gz | head -5 

node1	node2	label
Q35120	88859643	entity_count
Q99527517	74418826	entity_count
Q488383	73704542	entity_count
Q28813620	68227171	entity_count
zcat: error writing to output: Broken pipe


Jessica is curious about the data, so she writes a query to get the counts of different classes of film (Q11424). Jessica had been working with John, so she learned the trick to use the standard names for column headings so that she can use the output of previous queries as new graphs. She shudders to think that if she was using SPARQL she would have had to set up a new Wikidata SPARQL endpoint to be able to load her personal data in it, and to be extremely caeful to not make a mistake because deleting the data would have been a chore. Jessica had watched John make several mistakes when he was building the files for the names. John had simply fixed the queries and re-run the other queries that depended on the data he had just fixed.

In [30]:
!$kypher -i p279star -i labels -i "$OUT"/class.count.tsv.gz \
--match ' \
    p279star: (class)-[]->(:Q11424), \
    count: (class)-[]->(count), \
    labels: (class)-[]->(class_label)' \
--return 'class as class, class_label as name, count as count' \
--order-by 'cast(count, integer) desc' \
--limit 10 \
| column -ts $'\t'

[2021-07-18 15:49:30 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1" "_aLias.class", graph_4_c3."node2" "_aLias.name", graph_33_c2."node2" "_aLias.count"
     FROM graph_33 AS graph_33_c2
     INNER JOIN graph_4 AS graph_4_c3, graph_5 AS graph_5_c1
     ON graph_5_c1."node1" = graph_33_c2."node1"
        AND graph_5_c1."node1" = graph_4_c3."node1"
        AND graph_5_c1."node2" = ?
     ORDER BY CAST("_aLias.count" AS integer) DESC
     LIMIT ?
  PARAS: ['Q11424', 10]
---------------------------------------------
        1.09 real         0.87 user         0.20 sys
class      name                      count
Q11424     'film'@en                 314889
Q24862     'short film'@en           33733
Q506240    'television film'@en      17310
Q226730    'silent film'@en          17131
Q20667187  'silent short film'@en    16302
Q202866    'animated film'@en        9019
Q17517379  'animated short film'@en  4100
Q10590726  'video album'@en       

Jessica now has the statistics she needs to work on her feature for the entity resolution algorithm. 

## Extract new graphs from Wikidata

Bill is working on a project to find networks of researchers working on specific topics. He wants to use publication data to find relationships among authors using publications. Bill knows that he can get lots of publication data from Pubmed or Microsoft Academic graph, but wants to give Wikidata a try as he heard that Wikidata has close to 40 million publications, and that in Wikidata publications have links to other entities such as main subjects.

Bill decides that the simplest experiment to try first is to build a network of authors of publications in Wikidata: he wants to create a graph of people in Wikidata who authored papers, to put a link between two people if the coauthored a paper, and to add a qualifier with the count of papers they coauthored. He knows the computation is expensive as there are 40ish million papers in Wikidata, so the network will be large. He doesn't even try to write a SPARQL query because he knows it will time out. Bill downloads the KGTK files and decides to write his first query using only 2019 data so he doesn't have to wait so long if he makes a mistake.

### First do it for 2019 to debug the query

In [81]:
!$kypher -i p31 -i p279star -i items -i time -i labels \
--match '\
    p31: (pub)-[]->(class), \
    p279star: (class)-[]->(:Q591041), \
    time: (pub)-[:P577]->(pub_date), \
    items: (pub)-[:P50]->(author1), \
    items: (pub)-[:P50]->(author2), \
    labels: (author1)-[]->(author1_label)' \
--where 'author1 != author2 and kgtk_date_year(pub_date) = 2019' \
--return 'distinct author1 as node_x, "Pcoauthor" as relation, author2 as node_y, count(distinct pub) as count_publications, author1_label as `node1;label`' \
--order-by 'count_publications desc' \
-o "$TEMP"/coauthors.2019.id.tsv.gz

[2021-07-18 17:39:20 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c4."node2" "_aLias.node_x", ? "_aLias.relation", graph_8_c5."node2" "_aLias.node_y", count(DISTINCT graph_2_c1."node1") "_aLias.count_publications", graph_4_c6."node2" "_aLias.node1;label"
     FROM graph_14 AS graph_14_c3
     INNER JOIN graph_2 AS graph_2_c1, graph_4 AS graph_4_c6, graph_5 AS graph_5_c2, graph_8 AS graph_8_c4, graph_8 AS graph_8_c5
     ON graph_2_c1."node1" = graph_14_c3."node1"
        AND graph_2_c1."node1" = graph_8_c4."node1"
        AND graph_2_c1."node1" = graph_8_c5."node1"
        AND graph_2_c1."node2" = graph_5_c2."node1"
        AND graph_8_c4."node2" = graph_4_c6."node1"
        AND graph_14_c3."label" = ?
        AND graph_5_c2."node2" = ?
        AND graph_8_c4."label" = ?
        AND graph_8_c5."label" = ?
        AND ((graph_8_c4."node2" != graph_8_c5."node2") AND (kgtk_date_year(graph_14_c3."node2") = ?))
     GROUP BY "_aLias.node_x

In [82]:
!zcat < "$TEMP"/coauthors.2019.id.tsv.gz | head

node_x	relation	node_y	count_publications	node1;label
Q104625960	Pcoauthor	Q104626213	117	'Secundino López Puente'@en
Q104625960	Pcoauthor	Q42121517	117	'Secundino López Puente'@en
Q104625960	Pcoauthor	Q46702124	117	'Secundino López Puente'@en
Q104625960	Pcoauthor	Q57221019	117	'Secundino López Puente'@en
Q104625960	Pcoauthor	Q57235422	117	'Secundino López Puente'@en
Q104625960	Pcoauthor	Q62593499	117	'Secundino López Puente'@en
Q104625960	Pcoauthor	Q62607742	117	'Secundino López Puente'@en
Q104625960	Pcoauthor	Q80042771	117	'Secundino López Puente'@en
Q104626213	Pcoauthor	Q104625960	117	'Roberto Edoardo Villa'@en
zcat: error writing to output: Broken pipe


Bill wants to sanity check his data so he looks up the first person in Google Scholar and finds that Secundino López Puente has many publications in 2019. Looks like the query is working fine.

### Build the network for all authors

Bill removes the year restriction and runs the query for the full data. The query for a single year took close to 10 minutes, so Bill decides to leave the query running overnight.

In [16]:
!$kypher -i p31 -i p279star -i items -i time -i labels \
--match '\
    p31: (pub)-[]->(class), \
    p279star: (class)-[]->(:Q591041), \
    time: (pub)-[:P577]->(pub_date), \
    items: (pub)-[:P50]->(author1), \
    items: (pub)-[:P50]->(author2)' \
--where 'author1 != author2' \
--return 'distinct author1 as node_x, "Pcoauthor" as relation, author2 as node_y, count(distinct pub) as count_publications' \
--order-by 'count_publications desc' \
-o "$TEMP"/coauthors.2019.tsv.gz

[2021-07-18 11:26:17 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c4."node2" "_aLias.node_x", ? "_aLias.relation", graph_8_c5."node2" "_aLias.node_y", count(DISTINCT graph_2_c1."node1") "_aLias.count_publications"
     FROM graph_14 AS graph_14_c3
     INNER JOIN graph_2 AS graph_2_c1, graph_5 AS graph_5_c2, graph_8 AS graph_8_c4, graph_8 AS graph_8_c5
     ON graph_2_c1."node1" = graph_14_c3."node1"
        AND graph_2_c1."node1" = graph_8_c4."node1"
        AND graph_2_c1."node1" = graph_8_c5."node1"
        AND graph_2_c1."node2" = graph_5_c2."node1"
        AND graph_14_c3."label" = ?
        AND graph_5_c2."node2" = ?
        AND graph_8_c4."label" = ?
        AND graph_8_c5."label" = ?
        AND ((graph_8_c4."node2" != graph_8_c5."node2") AND (kgtk_date_year(graph_14_c3."node2") = ?))
     GROUP BY "_aLias.node_x", "_aLias.relation", "_aLias.node_y"
     ORDER BY "_aLias.count_publications" DESC
  PARAS: ['Pcoauthor', 'P577', 

### Build a network of authors who authored papers about cancer

Bill is interested in cancer research, so he wants to build the same network but using only the papers about cancer. He knows Wikidata has an extensive class hiearchy, so he writes a query to peek at the hierarchy below the q-node for cancer.
He writes a query to retrieve subclasses of cancer.

In [17]:
!$kypher -i p279star -i labels \
--match '\
    p279star: (cancer_type)-[]->(:Q12078), \
    labels: (cancer_type)-[]->(cancer_type_label)' \
--return 'cancer_type as node1, cancer_type_label as node2' \
--limit 10

[2021-07-18 11:33:56 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1" "_aLias.node1", graph_4_c2."node2" "_aLias.node2"
     FROM graph_4 AS graph_4_c2
     INNER JOIN graph_5 AS graph_5_c1
     ON graph_5_c1."node1" = graph_4_c2."node1"
        AND graph_5_c1."node2" = ?
     LIMIT ?
  PARAS: ['Q12078', 10]
---------------------------------------------
node1	node2
Q101541302	'pulmonary artery intimal sarcoma'@en
Q101541613	'rectal small cell carcinoma'@en
Q101541672	'CIC-DUX4 sarcoma'@en
Q101541689	'colorectal large cell neuroendocrine carcinoma'@en
Q1016605	'Burkitt lymphoma'@en
Q102258467	'diffuse gastric cancer'@en
Q102293219	'luminal breast carcinoma B'@en
Q102293292	'skin meningioma'@en
Q102293358	'breast implant-associated anaplastic large cell lymphoma'@en
Q102293373	'salivary gland mucinous adenocarcinoma'@en
        1.27 real         0.86 user         0.18 sys


The results are promising, so Bill now incorporates the query for types of cancer into the query for building the coauthor network. He just needs to get the main subject of the paper using the `P921` property and test that the main subject is a subclass of cancer. He expects the query to be much faster because now it has strong restriction, so he gives it a try.

In [87]:
!$kypher -i p31 -i p279star -i items -i time -i labels \
--match '\
    p31: (pub)-[]->(class), \
    p279star: (class)-[]->(:Q591041), \
    time: (pub)-[:P577]->(pub_date), \
    items: (pub)-[:P50]->(author1), \
    items: (pub)-[:P50]->(author2), \
    items: (pub)-[:P921]->(cancer_type), \
    p279star: (cancer_type)-[]->(:Q12078), \
    labels: (author1)-[]->(author1_label), \
    labels: (author2)-[]->(author2_label)' \
--where 'author1 != author2' \
--return 'distinct author1 as node_x, "Pcoauthor" as relation, author2 as node_y, count(distinct pub) as count_publications, author1_label as `node1;label`, author2_label as `node2;label`' \
--order-by 'count_publications desc' \
-o "$TEMP"/coauthors.cancer.tsv.gz

[2021-07-18 17:57:40 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c4."node2" "_aLias.node_x", ? "_aLias.relation", graph_8_c5."node2" "_aLias.node_y", count(DISTINCT graph_2_c1."node1") "_aLias.count_publications", graph_4_c8."node2" "_aLias.node1;label", graph_4_c9."node2" "_aLias.node2;label"
     FROM graph_14 AS graph_14_c3
     INNER JOIN graph_2 AS graph_2_c1, graph_4 AS graph_4_c8, graph_4 AS graph_4_c9, graph_5 AS graph_5_c2, graph_5 AS graph_5_c7, graph_8 AS graph_8_c4, graph_8 AS graph_8_c5, graph_8 AS graph_8_c6
     ON graph_2_c1."node1" = graph_14_c3."node1"
        AND graph_2_c1."node1" = graph_8_c4."node1"
        AND graph_2_c1."node1" = graph_8_c5."node1"
        AND graph_2_c1."node1" = graph_8_c6."node1"
        AND graph_2_c1."node2" = graph_5_c2."node1"
        AND graph_8_c4."node2" = graph_4_c8."node1"
        AND graph_8_c5."node2" = graph_4_c9."node1"
        AND graph_8_c6."node2" = graph_5_c7."node1"
      

The query takes less than a minute and produces a network with close to half a million edges. Bill takes a peek to see what is in it, and now wonders whether he could have written the query in SPARQL and run it on the public SPARQL endpoint.

In [83]:
!zcat < "$TEMP"/coauthors.cancer.tsv.gz | wc

  456197 1824788 14594526


In [88]:
!zcat < "$TEMP"/coauthors.cancer.tsv.gz | head | column -ts $'\t'

zcat: error writing to output: Broken pipe
node_x     relation   node_y     count_publications  node1;label                   node2;label
Q60320900  Pcoauthor  Q60394812  396                 'Jorge Eduardo Cortes'@en     'Hagop Kantarjian'@en
Q60394812  Pcoauthor  Q60320900  396                 'Hagop Kantarjian'@en         'Jorge Eduardo Cortes'@en
Q60394812  Pcoauthor  Q66370727  236                 'Hagop Kantarjian'@en         'Susan O\'Brien'@en
Q66370727  Pcoauthor  Q60394812  236                 'Susan O\'Brien'@en           'Hagop Kantarjian'@en
Q40614280  Pcoauthor  Q60394812  186                 'Farhad Ravandi'@en           'Hagop Kantarjian'@en
Q60394812  Pcoauthor  Q40614280  186                 'Hagop Kantarjian'@en         'Farhad Ravandi'@en
Q60394812  Pcoauthor  Q66385413  180                 'Hagop Kantarjian'@en         'Guillermo Garcia-Manero'@en
Q66385413  Pcoauthor  Q60394812  180                 'Guillermo Garcia-Manero'@en  'Hagop Kantarjian'@en
Q60320900  Pcoa

Bill puts the first two names in Google and finds that they are famous and have publshied a lot together. Bill is happy to have a network with close to half a million edges that he can use to do interesting analyses.

## Query using large number of external identifiers

Abigail is working on a cultural heritage project, collaborating with the Getty Research Institute who gave her a file with 27 thousand ULAN identifiers. Abigail has a database indexed using VIAF identifiers, and wants to map her ULAN identifiers to VIAF identifiers so that she can use her database. She puts one of the ULAN identifiers in the Wikidata search box and discovers that Wikidata has both ULAN and VIAF identifiers for many artists. Abigail knows a little bit of SPARQL and easity figures out that it is easy to write a query to retrieve the VIAF identifier given a ULAN identifier. Her solution would require sending 27,000 queries to Wikidata, which would involve writing a Python script.

In [91]:
!wc "$OUT"/ulan.tsv

   27415   27415  356389 /Users/pedroszekely/Downloads/kypher/wd-workshop/ulan.tsv


Her colleague Bill tells her that she can easily solve the problem using KGTK query. The only thing she needs to do is to rename the header of her file with identifiers to `node1` and write a Kypher query.

In [19]:
!$kypher -i items -i external_ids -i labels -i "$OUT"/ulan.tsv \
--match '\
    ulan: (ulan_id)-[]->(), \
    external_ids: (viaf_id)<-[:P214]-(artist)-[:P245]->(ulan_id), \
    labels: (artist)-[]->(artist_label)' \
--return 'artist as qnode, viaf_id as viaf, ulan_id as ulan, artist_label as name' \
-o "$OUT"/ulan-to-viaf.tsv

[2021-07-18 11:36:12 query]: SQL Translation:
---------------------------------------------
  SELECT graph_28_c2."node1" "_aLias.qnode", graph_28_c2."node2" "_aLias.viaf", graph_32_c1."node1" "_aLias.ulan", graph_4_c4."node2" "_aLias.name"
     FROM graph_28 AS graph_28_c2
     INNER JOIN graph_28 AS graph_28_c3, graph_32 AS graph_32_c1, graph_4 AS graph_4_c4
     ON graph_28_c2."node1" = graph_28_c3."node1"
        AND graph_28_c2."node1" = graph_4_c4."node1"
        AND graph_32_c1."node1" = graph_28_c3."node2"
        AND graph_28_c2."label" = ?
        AND graph_28_c3."label" = ?
  PARAS: ['P214', 'P245']
---------------------------------------------
       16.20 real         1.44 user         2.45 sys


Abigail is thrilled to see that the query ran in less than 30 seconds and is curious to see the results. She got matches for 8,116 ULAN ids, which means that now she can get a lot of data from her database to

In [92]:
!wc "$OUT"/ulan-to-viaf.tsv

    8116   42730  443890 /Users/pedroszekely/Downloads/kypher/wd-workshop/ulan-to-viaf.tsv


In [20]:
!head "$OUT"/ulan-to-viaf.tsv | column -ts $'\t'

qnode       viaf         ulan         name
Q1000596    "20822441"   "500072302"  'Noémi Ferenczy'@en
Q1001063    "96418002"   "500099612"  'Olga Fialka'@en
Q100156272  "309815799"  "500335625"  'Gloria López Córdova'@en
Q100249806  "184467129"  "500040990"  'Alice Denniston Laughlin'@en
Q100250000  "63899160"   "500034511"  'Shirley L. Bolton'@en
Q100278786  "309815915"  "500336052"  'Winifred Casson'@en
Q100323915  "95510425"   "500332031"  'Claudia Müller'@en
Q100348403  "95887586"   "500033567"  'Priscilla Kepner Sage'@en
Q100377312  "233761"     "500288751"  'Cristina Castel-Branco'@en


## Combine Wikidata and DBpedia

## Validate property constraints in Wikidata